## Get API tokens

In [ ]:
import os
import urllib.parse

from dotenv import load_dotenv
import requests

In [ ]:
load_dotenv("../aws.env")

In [ ]:
api_url = urllib.parse.urljoin(os.environ["DBAPI_URL"], os.environ["DBAPI_STAGE"])

In [ ]:
login_data = {
    "username": os.environ["FIRST_USER"],
    "password": os.environ["FIRST_USER_PASSWORD"],
}
r = requests.post(f"{api_url}/token", data=login_data)
tokens = r.json()
a_token = tokens["access_token"]
token_headers = {"Authorization": f"Bearer {a_token}"}

In [ ]:
# TODO? turn this into a decorator that can be used as a utility function,
# maybe taking a user prefix and providing token headers

## Get a document from kbase

In [ ]:
r = requests.get(f"{api_url}/documents/ids/?skip=0&limit=1", headers=token_headers)
doc_id = r.json()[0]["id"]

In [ ]:
r = requests.get(f"{api_url}/documents/{doc_id}", headers=token_headers)
doc = r.json()

In [ ]:
text = doc["parsed_text"]

## Generate summaries

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = AutoModel.from_pretrained("facebook/bart-large-cnn")

In [ ]:
inputs = bart_tokenizer([text], max_length=1024, return_tensors='pt')
summary_ids = bart_model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)

In [ ]:
# from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
# model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
# inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
# # Generate Summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [ ]:
from transformers import AutoModelWithLMHead

In [ ]:
t5_model = AutoModelWithLMHead.from_pretrained("t5-base")
t5_tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512)
outputs = t5_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

In [ ]:
outputs

In [ ]:
t5_tokenizer.convert_tokens_to_string(t5_tokenizer.convert_ids_to_tokens(outputs[0]))

In [ ]:
# TODO summarize everything that is not transcript